In [1560]:
import pandas as pd
import random
import numpy as np
from statistics import mean
from sklearn.preprocessing import MinMaxScaler
from sklearn.decomposition import PCA

random.seed(265)
df = pd.read_excel("country_information.xlsx")

In [1561]:
X = df[['gini_index', 'corruption_perceptions_index', 'freedom_house', 'hdi', 'press_freedom', 'democracy_economist', 'populism', 'effective_coverage_of_health_services_index', 'trust_in_news_media', 'trust_in_government', 'trust_in_science', 'colonized']].to_numpy()
Y = df['country']


In [1562]:
scaler = MinMaxScaler()
scaler.fit(X)
X = scaler.transform(X)


In [1563]:
df[['gini_index', 'corruption_perceptions_index', 'freedom_house', 'hdi', 'press_freedom', 'democracy_economist', 'populism', 'effective_coverage_of_health_services_index', 'trust_in_news_media', 'trust_in_government', 'trust_in_science', 'colonized']] = X

In [1564]:
df.to_excel('country_information_normalized.xlsx')

In [1565]:
#2.
X = np.asmatrix(X)

In [1566]:
def nearest(point, center): #calculate the nearest point, get the index from the center group number, and the distance
    distance = []
    for c in center:
        d = np.linalg.norm(point-c)
        distance.append(d)
    index = distance.index(min(distance))
    return index, min(distance)

In [1567]:
def k_meanpp(X, K): #initialize part
    random.seed(265)
    rn = random.randrange(0,len(df),1)
    center = np.array(X[rn])
    sum = 0
    
    #print(rn)
    for k in range(1, K):
        mindistance = []
        totaldist = 0 #sum of the closest distance for each point
        weight = []
        for x in X:
            index, mindist = nearest(x, center)#choose the nearest distance
            totaldist = totaldist + mindist**2
            mindistance.append(mindist)
        
            
        for i in range(len(X)):
            w = mindistance[i]**2/totaldist    #weights
            weight.append(w)
            
             
        centerindex = random.choices(range(len(X)), weights = weight)  
        center = np.append(center, X[centerindex], axis = 0)
        #print(centerindex)
        
    return center
        
        
        
                

In [1568]:

center = k_meanpp(X, 6)
print(center)

[[1.         0.42105263 1.         0.66666667 0.75       0.66666667
  0.5528537  0.4375     0.72972973 0.         0.21153846 1.        ]
 [0.28800554 0.59210526 1.         1.         0.75       0.66666667
  1.         0.59375    0.41481695 0.56884918 0.11538462 1.        ]
 [0.62231119 0.15789474 0.5        0.33333333 0.25       0.33333333
  0.42857143 0.34375    0.48341577 0.38581793 0.19230769 1.        ]
 [0.17571968 0.21052632 0.         0.33333333 0.25       0.
  0.46722867 0.234375   0.64844149 0.77768959 0.51923077 1.        ]
 [0.30331725 0.36842105 0.5        0.33333333 0.25       0.66666667
  0.28571429 0.234375   0.68059171 0.83076923 0.36538462 1.        ]
 [0.24557062 0.53947368 0.         1.         0.         0.
  0.56116116 0.5        0.67369056 0.63962377 0.5        0.        ]]


In [1569]:
def averagelist(X, center, K): #calculate the average value of each cluster, append them to a list
    avelist = []
    for k in range(K):
        pointlist = []
        for x in X:
            index, mindist = nearest(x, center)
            if index == k:
                #x = np.array(x)
                pointlist.append(x)
        average = np.average(pointlist, axis = 0)
        avelist.append(average)
        #print(type(center))
        
    return avelist

In [1570]:
print(averagelist(X, center, 6))

[array([[0.79244128, 0.45175439, 1.        , 0.77777778, 0.75      ,
        0.77777778, 0.5176179 , 0.5625    , 0.6764097 , 0.07847808,
        0.26923077, 1.        ]]), array([[0.2419597 , 0.68684211, 0.91111111, 0.98518519, 0.69444444,
        0.78518519, 0.67747661, 0.75655706, 0.50644692, 0.44850059,
        0.36623602, 0.82222222]]), array([[0.45428306, 0.26023392, 0.40740741, 0.44444444, 0.38888889,
        0.38271605, 0.49959534, 0.42824074, 0.45829264, 0.47349159,
        0.19444444, 1.        ]]), array([[0.28185582, 0.265625  , 0.015625  , 0.43951602, 0.1796875 ,
        0.03269076, 0.50344673, 0.359375  , 0.51880224, 0.6787235 ,
        0.31754652, 0.90625   ]]), array([[0.28318519, 0.30858159, 0.63888889, 0.39988329, 0.47145436,
        0.49748941, 0.48040638, 0.23199215, 0.57779849, 0.7799202 ,
        0.27224494, 1.        ]]), array([[0.38011286, 0.32894737, 0.        , 0.91666667, 0.125     ,
        0.08333333, 0.38084597, 0.5625    , 0.51041084, 0.39850842,
        

In [1571]:
def update(X, center, K): #update if center doesn't equal to average, stop when converge. This is the Kmean part
    aver = False
    center = np.array(center)
     
    while aver == False:
        avelist = np.array(averagelist(X, center, K))
        avelist = avelist.reshape(6,1*12)

       
        if  (np.absolute(avelist - center) ==0).all() == True:
            aver = True
        else:
            center = avelist
            
            
    #print(center)
    return center

In [1572]:
center = update(X, center, 6)

In [1573]:
#3.
def label(X, center):#get the label of each point
    labellist = []

    for x in X:
        index, mindist = nearest(x, center)
        label = index + 1
        labellist.append(label)
           
        
    return(labellist)
    
    #return labellist
          
            

In [1574]:
labellist = label(X, center)
print(labellist)

[6, 3, 4, 1, 2, 1, 4, 4, 2, 3, 3, 1, 1, 5, 4, 4, 2, 1, 4, 3, 4, 1, 1, 4, 1, 2, 3, 4, 3, 4, 3, 1, 6, 3, 2, 1, 4, 5, 3, 2, 5, 1, 3, 5, 5, 1, 3, 2, 5, 3, 6, 4, 2, 2, 2, 5, 2, 4, 4, 5, 3, 3, 4, 1, 3, 4, 1, 5, 5, 3, 5, 3, 3, 3, 3, 5, 4, 5, 2, 2, 5, 4, 3, 4, 5, 5, 3, 3, 3, 1, 2, 4, 1, 6, 4, 6, 5, 3, 3, 1, 1, 1, 6, 2, 3, 4, 2, 2, 4, 1, 4, 5, 3, 5, 1, 6, 5, 4, 3, 4, 2, 1, 1, 4, 5, 4, 4, 4, 4]


In [1575]:
df['kmeans_label'] = labellist

In [1576]:
pca = PCA(n_components=2, random_state = 265)
X = np.asarray(X) 
X = pca.fit_transform(X)


In [1577]:
df['pca_dim_1'] = X[:,0]
df['pca_dim_2'] = X[:,1]
print(df)

         country  gini_index  corruption_perceptions_index  freedom_house  \
0    Afghanistan    0.415418                      0.092105            0.0   
1        Albania    0.239518                      0.315789            0.5   
2        Algeria    0.096609                      0.315789            0.0   
3      Argentina    0.487058                      0.394737            1.0   
4      Australia    0.270142                      0.855263            1.0   
..           ...         ...                           ...            ...   
124      Vanuatu    0.351804                      0.407895            1.0   
125    Venezuela    0.535545                      0.039474            0.0   
126      Vietnam    0.303317                      0.315789            0.0   
127        Yemen    0.328837                      0.039474            0.0   
128     Zimbabwe    0.675902                      0.157895            0.0   

          hdi  press_freedom  democracy_economist  populism  \
0    0.00000

In [1578]:
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns

mpl.rcParams['figure.dpi'] = 600
plt.rcParams['figure.figsize'] = (20.0, 10.0)
plt.rcParams['font.family'] = "serif"
plt.rcParams['font.size'] = 10

df = df
pal = sns.color_palette("Paired")[:len(set(df['kmeans_label']))]
p1 = sns.scatterplot(x="pca_dim_1", y='pca_dim_2', hue='kmeans_label', palette = pal, data=df, s=250, alpha=0.7, legend=False)

#For each point, we add a text inside the bubble
for line in range(0,df.shape[0]):
     p1.text(df.pca_dim_1[line], df.pca_dim_2[line], df.country[line], horizontalalignment='left', size='medium', color='black', weight='semibold')

plt.suptitle('Two-Dimensional Map of Countries (PCA)', fontsize=36)
plt.xlabel('PCA - Dimension 1', fontsize=24)
plt.ylabel('PCA - Dimension 2', fontsize=24)

from scipy import interpolate
from scipy.spatial import ConvexHull

for i in df.kmeans_label.unique():
    # get the convex hull
    points = df[df.kmeans_label == i][['pca_dim_1', 'pca_dim_2']].values
    hull = ConvexHull(points)
    x_hull = np.append(points[hull.vertices,0],
                       points[hull.vertices,0][0])
    y_hull = np.append(points[hull.vertices,1],
                       points[hull.vertices,1][0])
    
    # interpolate
    dist = np.sqrt((x_hull[:-1] - x_hull[1:])**2 + (y_hull[:-1] - y_hull[1:])**2)
    dist_along = np.concatenate(([0], dist.cumsum()))
    spline, u = interpolate.splprep([x_hull, y_hull], 
                                    u=dist_along, s=0)
    interp_d = np.linspace(dist_along[0], dist_along[-1], 50)
    interp_x, interp_y = interpolate.splev(interp_d, spline)
    # plot shape
    plt.fill(interp_x, interp_y, '--', c=pal[i-1], alpha=0.2)
    

plt.grid()
plt.show()